In [1]:
%%file cpp_regularized_linear_regression.hpp
double* coordinate_descent(double *data_x, double *data_y, double *beta, int num_samples, int num_features);

Overwriting cpp_regularized_linear_regression.hpp


In [2]:
%%file cpp_regularized_linear_regression.cpp
#include <iostream>
#include <cmath>
#define ARMA_DONT_USE_WRAPPER
#include <armadillo>


using namespace std;
using namespace arma;

double S(double z, double gamma) {  
    
    return 100.5;
    
    if (z >= 0){
        if (z - gamma > 0){
            return z - gamma;
        }
        else {
            return 0;
        }
    }
    else {
        if (z + gamma > 0){
            return z + gamma;
        }
        else {
            return 0;
        }
    }
}
    

double* coordinate_descent(double *data_x, double *data_y, double *beta, int num_samples, int num_features){
    double alpha=1, L1_ratio=0.5;
    int itr, j;
    mat X;
    vec y, b, b_new;
    X = mat(data_x, num_samples, num_features);
    y = vec(data_y, num_samples);
    b = vec(beta, num_features);
    
    for(itr=0; itr<1000; itr++){
        for(j=0; j<num_features; j++){
            b_new[j] = S(1.0 * arma::dot (X.col(j), y - (arma::dot (X, b) - (X.col(j) * b(j)))), 1*0.5) / (1 + alpha*(1-L1_ratio));
        }
        
        b = b_new;
    }
    
    for(int i=0;i<num_features;i++){
        beta[i] = b[i];
    }
    
    return beta;
    
}



Overwriting cpp_regularized_linear_regression.cpp


In [3]:
%%file cy_regularized_linear_regression.pxd

cdef extern from "cpp_regularized_linear_regression.hpp":
    double* coordinate_descent(double *data_x, double *data_y, double *beta, int num_samples, int num_features);

Overwriting cy_regularized_linear_regression.pxd


In [4]:
%%file cy_regularized_linear_regression.pyx
# distutils: language = c++
# distutils: sources = cpp_regularized_linear_regression.cpp

cimport cy_regularized_linear_regression
from libc.stdlib cimport malloc, free


def py_coord_desc(double[::1] data_x, double[::1] data_y, double[::1] beta_0, int num_samples, int num_features):
    cy_regularized_linear_regression.coordinate_descent(&data_x[0], &data_y[0], &beta_0[0], num_samples, num_features)

Overwriting cy_regularized_linear_regression.pyx


In [8]:
%%file setup.py

from distutils.core import setup, Extension
from Cython.Build import cythonize
import numpy as np

ext = Extension("cy_regularized_linear_regression",
                sources=["cy_regularized_linear_regression.pyx","cpp_regularized_linear_regression.cpp"],
                libraries=["m", "armadillo", "lapack"],
                language=["c++"],
                extra_compile_args=["-std=c++11","-static"])

setup(name = "cy_regular_linreg",
      ext_modules = cythonize(ext))

Overwriting setup.py


In [9]:
! python setup.py clean
! python setup.py -q build_ext --inplace

running clean
removing 'build/temp.linux-x86_64-3.4' (and everything under it)
removing 'build'


In [10]:
import numpy as np
from sklearn import datasets
diabetes = datasets.load_diabetes()
data_X = diabetes.data
data_y = diabetes.target
num_samples, num_features = data_X.shape
data_X = data_X.reshape((num_samples*num_features,))

import cy_regularized_linear_regression


beta_0 = np.zeros(num_features)
print(beta_0)
cy_regularized_linear_regression.py_coord_desc(data_X, data_y, beta_0, num_samples, num_features)
print(beta_0)

ImportError: /opt/conda/lib/python3.4/site-packages/numpy/core/../../../.././libgfortran.so.3: version `GFORTRAN_1.4' not found (required by /usr/lib/liblapack.so.3)

In [ ]:
print('status')